In [1]:
%load_ext autoreload
%autoreload 2

## Doc2Vec

In [2]:
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from transformers.wl_graph_kernel_transformer import WLGraphKernelTransformer
from transformers.preprocessing_transformer import PreProcessingTransformer
from transformers.d2v_transformer import Doc2VecTransformer
import graph_helper
import dataset_helper
import wl
import os

for dataset_name in dataset_helper.get_all_available_dataset_names():
    if dataset_name != 'r8': continue
        
    X, Y = dataset_helper.get_dataset(dataset_name, use_cached= True)
    
    p = Pipeline([
        ('preprocessing', PreProcessingTransformer(only_nouns = True)),
        ('d2v', Doc2VecTransformer()),
        ('clf', sklearn.linear_model.PassiveAggressiveClassifier())
    ])
    
    param_grid = dict(
        d2v__embedding_size = [500],
        d2v__iterations = [10],
        d2v__infer_steps = [10],
        clf__n_iter = [100],
        clf__class_weight = ['balanced']
    )

    cv = sklearn.model_selection.StratifiedKFold(n_splits = 3, random_state= 42, shuffle= True)
    gscv = GridSearchCV(estimator = p, param_grid=param_grid, cv=cv, scoring = 'f1_macro', n_jobs=1, verbose = 11)
    gscv_result = gscv.fit(X, Y)
    print(gscv_result.best_estimator_, gscv_result.cv_results_)

KeyboardInterrupt: 

## Create co-occurrence graphs

In [ ]:
%%time
import dataset_helper
import preprocessing
import graph_helper
import cooccurrence
from joblib import Parallel, delayed
import networkx as nx

window_size = 1
min_length = 2

def process(X, Y):
    return graph_helper.convert_dataset_to_co_occurence_graph_dataset(X, Y, min_length = min_length, window_size = window_size, n_jobs = 1)

for dataset in dataset_helper.get_all_available_dataset_names():
    print("Creating co-occurence graphs for: {}".format(dataset))
    cache_file = dataset_helper.CACHE_PATH + '/dataset_graph_cooccurrence_{}_with-nouns_{}.npy'.format(window_size, dataset)
    X, Y = dataset_helper.get_dataset(dataset, preprocessed = False, use_cached=True, transform_fn=process, cache_file=cache_file)

## DeepWalk

In [ ]:
import deepwalk
from deepwalk import graph
from deepwalk import walks as serialized_walks
from gensim.models import Word2Vec
from deepwalk.skipgram import Skipgram
import dataset_helper
import graph_helper
import random
from gensim.models import Word2Vec
import tsne
import matplotlib.pyplot as plt

max_memory_data_size = 1000000000
number_walks = 1000
representation_size = 64
seed = 0
undirected = True
vertex_freq_degree = False
walk_length = 60
window_size = 10
workers = 1
output = 'data/DUMP'

for dataset in dataset_helper.get_all_available_dataset_names():
    cache_file = dataset_helper.CACHE_PATH + '/dataset_graph_cooccurrence_{}.npy'.format(dataset)
    X, Y = dataset_helper.get_dataset(dataset, preprocessed = False, use_cached=True, transform_fn=graph_helper.convert_dataset_to_co_occurence_graph_dataset, cache_file=cache_file)
    break
    
models = []
for idx, g in enumerate(X):
    if idx == 3: break
    print('Graph: {:>4}'.format(idx))
    G = graph.from_networkx(g)

    print("Number of nodes: {}".format(len(G.nodes())))
    if len(G.nodes()) == 0:
        continue

    num_walks = len(G.nodes()) * number_walks

    print("Number of walks: {}".format(num_walks))

    data_size = num_walks * walk_length

    print("Data size (walks*length): {}".format(data_size))

    print("Walking...")
    walks = graph.build_deepwalk_corpus(G, num_paths=number_walks, path_length=walk_length, alpha=0, rand=random.Random(seed))
    print("Training...")
    model = Word2Vec(walks, size=representation_size, window=window_size, min_count=0, workers=workers)

    #model.wv.save_word2vec_format(output)
    models.append(model)
print('Finished')

## tSNE

In [ ]:
for model in models:
    print('Next')
    vectors = tsne.get_tsne_embedding(model)
    tsne.plot_embedding(model, vectors)
    plt.show()

## Test WL phi computation

In [ ]:
from joblib import Parallel, delayed

from time import time
phi_list_train_last = phi_list_train[-1]
test_graphs = train[:100]

def test_graph(idx, a):
    topic, graph = a
    #if idx % 1 == 0: print('{:>8}/{}'.format(idx, len(test_graphs)))
    phi_train = wl.compute_phi(graph, phi_list_train_last.shape, label_lookup_train, label_counters_train, h = 1)
    for i, (real, new) in enumerate(zip(phi_list_train, phi_train)):
        real = real[:,idx]
        new = lil_matrix(new.reshape(-1,1))
        if not np.array_equiv(real.nonzero()[0], new.nonzero()[0]):
            print('Phi not equal', i, 'Real', real, '\nNew\n', new)
    print('Finished: {}'.format(idx))
    
mats = Parallel(n_jobs=2)(delayed(test_graph)(*d) for d in list(enumerate(test_graphs)))

for idx, (topic, graph) in enumerate(test_graphs):
    break
    if idx % 1 == 0: print('{:>8}/{}'.format(idx, len(test_graphs)))
    phi_train = wl.compute_phi(graph, phi_list_train_last.shape, label_lookup_train, label_counters_train, h = 1)
    for i, (real, new) in enumerate(zip(real_, phi_train)):
        real = real[:,idx]
        new = lil_matrix(new.reshape(-1,1))
        if not np.array_equiv(real.nonzero()[0], new.nonzero()[0]):
            print('Phi not equal', i, 'Real', real, '\nNew\n', new)
            break


In [ ]:
import functools
import wl
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix, csr_matrix, vstack

def get_all_nodes(gs):
    return functools.reduce(lambda acc, x: acc | set(x), gs, set())

def get_wl_args(graphs):
    adjs = [nx.adjacency_matrix(g).toarray() for g in graphs]
    nodes = [g.nodes() for g in graphs]
    return adjs, nodes


g1 = nx.DiGraph()
g1.add_edge('A', 'B')
g1.add_edge('B', 'C')

g2 = nx.DiGraph()
g2.add_edge('A', 'B')
g2.add_edge('B', 'C')
g2.add_edge('B', 'D')

g3 = nx.DiGraph()
g3.add_edge('E', 'F')
g3.add_node('G')
all_graphs = (g1, g2, g3)

DEBUG = False
H = 10

all_nodes = get_all_nodes((g1, g2, g3))

adjs, nodes = get_wl_args((g1, g2))
K_1_2, phi_1_2, label_lookups_1_2, label_counters_1_2 = wl.WL_compute(ad_list=adjs, node_label=nodes, all_nodes=all_nodes, h = H, DEBUG=DEBUG)
adjs, nodes = get_wl_args((g1, g2, g3))
K_1_2_3, phi_1_2_3, label_lookups_1_2_3, label_counters_1_2_3 = wl.WL_compute(ad_list=adjs, node_label=nodes, all_nodes=all_nodes, h = H, DEBUG=DEBUG)

TARGET_GRAPH = g3
K_1_2_3_test, phi_1_2_3_test = wl.WL_compute_new(
    ad_list=[nx.adjacency_matrix(TARGET_GRAPH).toarray()],
    node_label=[TARGET_GRAPH.nodes()],
    label_counters_prev = label_counters_1_2,
    all_nodes= all_nodes,
    h = H,
    k_prev = np.copy(K_1_2),
    phi_prev = np.copy(phi_1_2),
    label_lookups_prev = np.copy(label_lookups_1_2)
)

phi_3_test = wl.compute_phi(g3, phi_1_2_3[0].shape, label_lookups_1_2_3, label_counters_1_2_3, h = H)
for idx, (real, new) in enumerate(zip(phi_1_2_3, phi_3_test)):
    real = real[:,2]
    new = lil_matrix(new.reshape(-1,1))
    if not np.array_equiv(real.todense(), new.todense()):
        print('Phi not equal', idx)

if 0 == 1:
    for i, (a, b) in enumerate(zip(phi_1_2_3_test, phi_1_2_3)):
        if not np.array_equiv(a - b.todense(), np.zeros(b.shape, dtype = np.int32)):
            print("\tPhi different! {}".format(i))
            print(np.argwhere((a - b) != 0))

    for i, (a, b) in enumerate(zip(K_1_2_3_test, K_1_2_3)):
        if not np.array_equal(a, b):
            print(np.argwhere((a - b) != 0))
            print("\tK different! {}".format(i))


## Results

In [ ]:
from glob import glob
import pickle

for file in glob('data/results/*.npy'):
    with open(file, 'rb') as f:
        results = pickle.load(f)
    if 'ling-spam' not in file: continue
    print('#### {}:\n\t{}'.format(file.split('/')[-1], results['mean_test_score'][0]))

## Batched phi calculation

In [ ]:
import dataset_helper
import graph_helper
import os
import wl
import numpy as np
import networkx as nx

def get_size(file):
    return os.path.getsize(file)

for graph_cache_file in sorted(dataset_helper.get_all_cached_graph_datasets(), key = lambda x: get_size(x)):
    X, Y = dataset_helper.get_dataset_cached(cache_file=graph_cache_file)
    
    graphs = [g for g in X[:2000] if nx.number_of_nodes(g) > 0 and nx.number_of_edges(g) > 0]
    adjs, nodes = graph_helper.get_wl_args(graphs)
    all_node_labels = graph_helper.get_all_node_labels(X)
    print("finished")
    K, phi_list, label_lookups, label_counters = wl.WL_compute(adjs, nodes, 1, all_node_labels, compute_k=False )

    #K_new, phi_list_new, label_lookups_new, label_counters_new = wl.WL_compute([adjs[-1]], [nodes[-1]], 1, all_node_labels, compute_k=False, initial_label_counters= label_counters, initial_label_lookups= label_lookups)
    #print(np.array_equiv(phi_list[-1][:,-1].nonzero()[0], phi_list_new[-1][:,-1].nonzero()[0]))
    break

In [ ]:
from time import time
h = 3
start_1 = time()
phi_lists, current_label_lookups, current_label_counters = wl.WL_compute_batched(adjs=adjs, node_label=nodes, batch_size=1000, all_nodes = all_node_labels, compute_k = False, h = h, DEBUG = True)
start_2 = time()
K, phi_list, label_lookups, label_counters = wl.WL_compute(adjs, nodes, all_nodes = all_node_labels, h = h, compute_k=False , DEBUG = True)
end = time()
print('First:\t{}\nSecond:\t{}'.format(start_2 - start_1, end - start_2))
print(phi_lists[-1].shape, len(adjs))

In [ ]:
#print(phi_lists[-1].nonzero(), phi_list[-1].nonzero())
#print(np.array_equiv(phi_lists[-1].nonzero(), phi_list[-1].nonzero()))
already_checked = set()
for idx in zip(*phi_lists[-1].nonzero()):
    val_a = phi_lists[-1][idx]
    val_b = phi_list[-1][idx]
    already_checked |= set(idx)
    if val_a != val_b:
        print("?")
        
for idx in zip(*phi_list[-1].nonzero()):
    val_a = phi_lists[-1][idx]
    val_b = phi_list[-1][idx]
    if val_a != val_b:
        print("?")

#    print(row, col)
#np.any(phi_lists[-1].nonzero()phi_list[-1].nonzero())

In [167]:
import networkx as nx
import sympy
import numpy as np
from scipy.sparse import lil_matrix, coo_matrix
from collections import defaultdict

def get_prime_range(start, end):
    return list(sympy.primerange(start, end + 1))

primes_arguments_required = [2, 3, 7, 19, 53, 131, 311, 719, 1619, 3671, 8161, 17863, 38873, 84017, 180503, 386093, 821641, 1742537, 3681131, 7754077, 16290047]

def fast_wl_compute(graphs, h = 1, label_lookup = {}, primes_arguments_required = primes_arguments_required):
    num_labels = len(label_lookup.keys())
    num_graphs = len(graphs)
    
    primes_needed = primes_arguments_required[int(np.ceil(np.log2(num_labels)) + 2)]
    p = get_prime_range(1, primes_needed)
    log_primes = np.log(p)
    label_counters = []
    label_lookups = []
    labels = [sorted(g.nodes()) for g in graphs]
    adjs = [nx.adjacency_matrix(graph, nodelist=labels_) for graph, labels_ in zip(graphs, labels)]
    phi_shape = (sum(len(x) for x in labels), num_graphs)
    
    # Uncomment when gram/kernel matrix is wanted
    #K = np.zeros((num_graphs, num_graphs), dtype = np.uint32)
    phi_lists = []
    for i in range(h):
        label_lookup = {}
        label_counter = 1
        phis = []
        phi = lil_matrix(phi_shape, dtype = np.uint8)
        for idx, (labels_, A) in enumerate(zip(labels, adjs)):
            signatures = np.round((labels_ + A * log_primes[np.array(labels_) - 1]), decimals=10)
            new_labels = np.zeros(len(signatures), dtype = np.uint8)
            for idx_, signature in enumerate(signatures):
                if signature not in label_lookup:
                    label_lookup[signature] = label_counter
                    label_counter += 1
                new_labels[idx_] = label_lookup[signature]
            labels[idx] = new_labels
            phis.append(new_labels)
            phi[new_labels - 1, idx] = np.ones(phi[new_labels - 1, idx].shape, dtype = np.uint8)
        phi_lists.append(phi)
        # Uncomment when gram/kernel matrix is wanted
        #K += phi.T.dot(phi)
    return phi_lists


def relabel_graphs(graphs, label_lookup = {}):
    label_counter = 1
    for graph in graphs:
        current_counter = sorted(label_lookup.keys())
        for label in sorted(graph.nodes()):
            if label not in label_lookup:
                label_lookup[label] = label_counter
                label_counter += 1
        nx.relabel_nodes(graph, {k: v for k, v in label_lookup.items() if k in graph}, copy = False)
    return label_lookup
    
g1 = nx.Graph()
g1.add_edge('A', 'B')
g1.add_edge('A', 'C')

g2 = g1.copy()

g3 = nx.Graph()
g3.add_edge('A', 'D')

graphs = [g1, g2, g3]
label_lookup = relabel_graphs(graphs)
new_labels = fast_wl_compute(graphs, h = 10, label_lookup=label_lookup)
print(new_labels[-1].todense())

[[30 30  0]
 [30 30  0]
 [ 0  0 20]]
[[1 1 0]
 [1 1 0]
 [1 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]]
